In [1]:
#ATHLETES

from bs4 import BeautifulSoup
import requests
import json
import re
import time

pays_dict = {
    'afghanistan': 'af',
    'albania': 'al',
    'algeria': 'dz',         
    'argentina': 'ar',
    'armenia': 'am',
    'australia': 'au',
    'australia / new zealand (australasia)': 'au/nz',
    'austria': 'at',
    'austria / united states of amerika': 'at/us',
    'azerbaijan': 'az',
    'bahamas': 'bs',
    'bahrain': 'bh',
    'barbados': 'bb',
    'belarus': 'by',
    'belgium': 'be',
    'bermuda': 'bm',
    'bohemia': 'cz',
    'bohemia / great britain': 'cz/gb',
    'botswana': 'bw',
    'brazil': 'br',
    'bulgaria': 'bg',
    'burkina-faso': 'bf',
    'burundi': 'bi',
    'cameroon': 'cm',         
    'canada': 'ca',
    'cape verde': 'cv',
    'chile': 'cl',
    'china': 'cn',
    'chinese taipei': 'tw',
    'colombia': 'co',
    'commonwealth of independent states': 'cis',
    'costa rica': 'cr',
    'cote divoire': 'ci',
    'croatia': 'hr',
    'cuba': 'cu',
    'cyprus': 'cy',
    'czech republic': 'cz',
    'czechoslovakia': 'cs',
    'denmark': 'dk',
    'djibouti': 'dj',
    'dominican': 'dm',
    'dominican republic': 'do',
    'ecuador': 'ec',
    'egypt': 'eg',
    'eritrea': 'er',
    'estonia': 'ee',
    'ethiopia': 'et',
    'federal republic of germany': 'de',
    'fiji': 'fj',
    'finland': 'fi',
    'france': 'fr',
    'france / great britain': 'fr/gb',
    'france / great britain / ireland': 'fr/gb/ie',
    'gabon': 'ga',
    'georgia': 'ge',
    'german democratic republic': 'dd',
    'germany': 'de',
    'ghana': 'gh',
    'great britain': 'gb',
    'great britain / australia': 'gb/au',
    'great britain / france / united states of america': 'gb/fr/us',
    'great britain / ireland': 'gb/ie',
    'great britain / ireland / germany': 'gb/ie/de',
    'greece': 'gr',
    'greece / belgium': 'gr/be',
    'grenada': 'gd',
    'guatemala': 'gt',
    'guyana': 'gy',
    'haiti': 'ht',
    'hong kong': 'hk',
    'hungary': 'hu',
    'iceland': 'is',
    'independent olympic participants': 'iop',
    'india': 'in',
    'individual neutral athletes': 'ina',
    'individual olympic athletes': 'ioa',
    'indonesia': 'id',
    'iran': 'ir',
    'iraq': 'iq',
    'ireland': 'ie',
    'israel': 'il',
    'italy': 'it',
    'jamaica': 'jm',
    'japan': 'jp',
    'japan / korea': 'jp/kr',
    'jordan': 'jo',         
    'kazakhstan': 'kz',
    'kenya': 'ke',
    'kosovo': 'xk',
    'kuwait': 'kw',
    'kyrgyzstan': 'kg',
    'latvia': 'lv',
    'lebanon': 'lb',
    'liechtenstein': 'li',
    'lithuania': 'lt',
    'luxembourg': 'lu',       
    'luxembourg / france': 'lu/fr',
    'malaysia': 'my',
    'mauritius': 'mu',
    'mexico': 'mx',
    'moldova': 'md',
    'mongolia': 'mn',
    'montenegro': 'me',
    'morocco': 'ma',         
    'mozambique': 'mz',
    'namibia': 'na',
    'netherlands': 'nl',
    'netherlands / france': 'nl/fr',
    'netherlands antilles': 'an',
    'new zealand': 'nz',
    'niger': 'ne',
    'nigeria': 'ng',
    'north korea': 'kp',     
    'north-macedonia': 'mk',
    'norway': 'no',
    'pakistan': 'pk',
    'panama': 'pa',
    'paraguay': 'py',
    'peru': 'pe',
    'philippines': 'ph',
    'poland': 'pl',
    'portugal': 'pt',
    'puerto rico': 'pr',     
    'qatar': 'qa',        
    'refugee olympic team': 'rot',
    'romania': 'ro',
    'russia': 'ru',
    'russian olympic committee': 'roc',
    'saint lucia': 'lc',     
    'samoa': 'ws',
    'san marino': 'sm',   
    'saudi arabia': 'sa',     
    'senegal': 'sn',
    'serbia': 'rs',
    'serbia montenegro': 'cs',
    'singapore': 'sg',
    'slovakia': 'sk',
    'slovenia': 'si',
    'smyrna': 'tr',
    'south africa': 'za',
    'south korea': 'kr',
    'soviet union': 'su',
    'spain': 'es',
    'sri lanka': 'lk',
    'sudan': 'sd',
    'suriname': 'sr',
    'sweden': 'se',
    'sweden / denmark': 'se/dk',
    'switzerland': 'ch',
    'syria': 'sy',           
    'tajikistan': 'tj',      
    'tanzania': 'tz',
    'thailand': 'th',
    'thessalonika': 'gr',
    'togo': 'tg',
    'tonga': 'to',
    'trinidad & tobago': 'tt',
    'tunisia': 'tn',         
    'turkey': 'tr',
    'turkmenistan': 'tm',
    'uganda': 'ug',          
    'ukraine': 'ua',
    'united arab emirates': 'ae', 
    'united states of america': 'us',
    'united states of america / cuba': 'us/cu',
    'united states of america / france': 'us/fr',
    'united states of america / germany': 'us/de',
    'united states of america / great britain': 'us/gb',
    'united states of america / mexico': 'us/mx',
    'uruguay': 'uy',       
    'uzbekistan': 'uz',      
    'venezuela': 've',
    'vietnam': 'vn',
    'virgin islands': 'vi',
    'west indies federation': 'wif',
    'yugoslavia': 'yu',
    'zambia': 'zm',
    'zimbabwe': 'zw'
}

# Création d'un dictionnaire inversé pour recherche par code
codes_pays_dict = {v: k for k, v in pays_dict.items()}

def extract_basic_info(person, pays_dict):
    """Extrait les informations de base d'un élément HTML d'athlète"""

    link = person.get('href')
    athlete_id = None
    if link:
        id_match = re.search(r'/olympic-athlete/[^/]+/(\d+)', link)
        if id_match:
            athlete_id = id_match.group(1)
    
    # Extraction du prénom et du nom
    prenom_div = person.select_one('div.vn')
    nom_div = person.select_one('div.n')
    prenom = prenom_div.get_text(strip=True) if prenom_div else ""
    nom = nom_div.get_text(strip=True) if nom_div else ""
    
    # Extraction du pays
    flag_img = person.select_one('img[src*="flagge"]')
    pays = flag_img.get('alt') if flag_img and flag_img.get('alt') else ""
    
    if not pays and flag_img and flag_img.get('src'):
        src = flag_img.get('src')
        pays_code = src.split('/')[-1].split('.')[0]
        pays = codes_pays_dict.get(pays_code.lower(), pays_code)
    
    # Extraction du sexe
    sexe = ""
    if person.select_one('svg.female'):
        sexe = 'female'
    elif person.select_one('svg.male'):
        sexe = 'male'
    
    user = {
        'id': athlete_id,
        'prenom': prenom,
        'nom': nom,
        'pays': pays,
        'sexe': sexe,
        'profile_url': f"https://olympics-statistics.com{link}" if link else "",
        'medailles': {
            'nombre': 0,
            'types': [],
            'sport': ""
        }
    }
    
    return user


def extract_medal_info(profile_url):
    """Extrait les informations de médailles depuis l'URL du profil d'un athlète"""
    try:
        response = requests.get(profile_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        

        nombre_medailles = 0
        types_medailles = []
        sport = ""
        

        medals = soup.select('.deck .medaille.visible')
        nombre_medailles = len(medals)
        
        for medal in medals:

            medal_div = medal.select_one('.the-medal')
            if medal_div and medal_div.has_attr('data-medal'):
                code = medal_div['data-medal']
                if code == '1':
                    types_medailles.append("or")
                elif code == '2':
                    types_medailles.append("argent")
                elif code == '3':
                    types_medailles.append("bronze")
            
            
            sport_div = medal.select_one('.m-sport')
            if sport_div:
                sport = sport_div.text.strip()
        
        return {
            'nombre': nombre_medailles,
            'types': types_medailles,
            'sport': sport
        }
    except Exception as e:
        print(f"Erreur lors de l'extraction des médailles pour {profile_url}: {str(e)}")
        return {
            'nombre': 0,
            'types': [],
            'sport': ""
        }

def get_athletes_from_page(url):

    athletes = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraire les cartes d'athlètes
        athletes_links = soup.select('a.card.athlet.visible')
        
        for athlete_link in athletes_links:
            # Extraire les informations de base
            athlete = extract_basic_info(athlete_link, pays_dict)
            athletes.append(athlete)
        

        pagination = soup.select('a.page-link')
        next_page_url = None
        
        for page_link in pagination:
 
            if page_link.text.strip() == '»' or page_link.text.strip().isdigit():
                href = page_link.get('href')
                if href and not href.startswith('http'):
                    next_page_url = f"https://olympics-statistics.com{href}"
                    break
        
        return athletes, next_page_url
    
    except Exception as e:
        print(f"Erreur lors de la récupération des athlètes depuis {url}: {str(e)}")
        return [], None

def main():

    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
              'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'special']
    

    all_athletes = []
    
    # Parcourir chaque lettre
    for letter in letters:
        print(f"Traitement des athlètes dont le nom commence par '{letter.upper()}'")
        

        if letter == 'special':
            current_url = f"https://olympics-statistics.com/olympic-athletes/{letter}"
        else:
            current_url = f"https://olympics-statistics.com/olympic-athletes/{letter}"
        
        page_number = 1
        
        # Parcourir toutes les pages pour cette lettre
        while current_url:
            print(f"  - Récupération de la page {page_number} pour '{letter.upper()}'")
            
            # Récupérer les athlètes de la page actuelle
            letter_athletes, next_page_url = get_athletes_from_page(current_url)
            
            print(f"    Nombre d'athlètes trouvés sur cette page: {len(letter_athletes)}")
            all_athletes.extend(letter_athletes)
            current_url = next_page_url
            page_number += 1
            time.sleep(1)
    
    print(f"Total d'athlètes trouvés: {len(all_athletes)}")

    
    
    for i, athlete in enumerate(all_athletes):
        if i % 10 == 0:  # Afficher la progression tous les 10 athlètes
            print(f"Traitement des médailles {i}/{len(all_athletes)}")
        
        if athlete['profile_url']:
           
            medals_info = extract_medal_info(athlete['profile_url'])
            athlete['medailles'] = medals_info
            
            
            time.sleep(0.5)
    

    output_file = "olympic_athletes_complet.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_athletes, f, ensure_ascii=False, indent=4)
    
    print(f"Informations sauvegardées pour {len(all_athletes)} athlètes dans {output_file}")

if __name__ == "__main__":
    main()

Traitement des athlètes dont le nom commence par 'A'
  - Récupération de la page 1 pour 'A'
    Nombre d'athlètes trouvés sur cette page: 1216
Traitement des athlètes dont le nom commence par 'B'
  - Récupération de la page 1 pour 'B'


KeyboardInterrupt: 

In [2]:
import json
import pandas as pd

try:
    with open('olympic_athletes_complet.json', 'r', encoding='utf-8') as f:
        athletes = json.load(f)
    print(f"Données chargées : {len(athletes)} athlètes trouvés")
except FileNotFoundError:
    print("Fichier athletes.json introuvable.")
    athletes = []

df = pd.DataFrame(athletes)
print(f"Aperçu initial - Colonnes: {df.columns.tolist()}")

def nettoyer_data():
    if len(df) == 0:
        return df
    
    # Extraction des médailles et du sport
    df['or'] = df['medailles'].apply(lambda x: x.get('types', []).count('or') if isinstance(x, dict) else 0)
    df['argent'] = df['medailles'].apply(lambda x: x.get('types', []).count('argent') if isinstance(x, dict) else 0)
    df['bronze'] = df['medailles'].apply(lambda x: x.get('types', []).count('bronze') if isinstance(x, dict) else 0)
    df['sport'] = df['medailles'].apply(lambda x: x.get('sport', "") if isinstance(x, dict) else "")
    
    # Nettoyage des données
    df['prenom'] = df['prenom'].fillna('')
    df['nom'] = df['nom'].fillna('')
    df['pays'] = df['pays'].fillna('Inconnu')
    df['nom_complet'] = df['prenom'] + ' ' + df['nom']
    

    pays_courts = df[df['pays'].str.len() == 2]['pays'].unique()
    if len(pays_courts) > 0:
        print(f"Codes pays courts détectés: {', '.join(pays_courts)}")
    

    df_clean = df.drop(columns=['medailles'])
    athletes_clean = df_clean.to_dict(orient='records')
    
    with open('athletes_complet_clean.json', 'w', encoding='utf-8') as f:
        json.dump(athletes_clean, f, ensure_ascii=False, indent=4)
    
    print(f"Données de {len(athletes_clean)} athlètes sauvegardées dans athletes_complet_clean.json")
    return df_clean

df_clean = nettoyer_data()
print("\nAperçu après nettoyage:")
print(df_clean.head())

Données chargées : 31843 athlètes trouvés

Aperçu avant nettoyage:
      id     prenom       nom                      pays    sexe  \
0  25024   Jennifer      Azzi  united states of america  female   
1  31553       Sera     Azuma                     japan  female   
2  27117  Okechukwu  Azubuike                   nigeria    male   
3  32171   Jeremiah       Azu             great britain    male   
4  27116    Jeremie      Azou                    france    male   

                                         profile_url  \
0  https://olympics-statistics.com/olympic-athlet...   
1  https://olympics-statistics.com/olympic-athlet...   
2  https://olympics-statistics.com/olympic-athlet...   
3  https://olympics-statistics.com/olympic-athlet...   
4  https://olympics-statistics.com/olympic-athlet...   

                                           medailles  
0  {'nombre': 1, 'types': ['or'], 'sport': 'Baske...  
1  {'nombre': 1, 'types': ['bronze'], 'sport': 'F...  
2  {'nombre': 1, 'types': ['